In [0]:
#!wget http://msvocds.blob.core.windows.net/coco2014/train2014.zip
#!wget http://msvocds.blob.core.windows.net/annotations-1-0-3/instances_train-val2014.zip 
#!wget http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip
#!wget http://msvocds.blob.core.windows.net/annotations-1-0-4/image_info_test2014.zip
#!wget http://msvocds.blob.core.windows.net/coco2014/val2014.zip

In [0]:
#!ls

In [0]:
#!unzip image_info_test2014.zip
#!unzip train2014.zip 
#!unzip val2014.zip
#!unzip instances_train-val2014.zip
#!unzip captions_train-val2014.zip

In [0]:
!ls

In [0]:
!pip install tqdm
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# from vgg16 import VGG16
import pickle
from tqdm import tqdm
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Merge, Activation, Flatten
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
import json
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image

In [0]:
images = 'train2014/'
# Contains all the images
img = glob.glob(images+'*.jpg')
img[:5]

In [0]:
len(img)

In [0]:
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

In [0]:
def preprocess(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    x = preprocess_input(x)
    return x

In [0]:
plt.imshow(np.squeeze(preprocess(img[0])))

In [0]:
model = InceptionV3(weights='imagenet')

In [0]:
from keras.models import Model

new_input = model.input
hidden_layer = model.layers[-2].output

model_new = Model(new_input, hidden_layer)

model_new.summary()

In [0]:
tryi = model_new.predict(preprocess(img[0]))

In [0]:
tryi.shape

In [0]:
def encode(image):
    image = preprocess(image)
    temp_enc = model_new.predict(image)
    temp_enc = np.reshape(temp_enc, temp_enc.shape[1])
    return temp_enc

In [0]:
encoding_train = {}
for img in tqdm(img):
    encoding_train[img[len(images):]] = encode(img)

In [0]:
with open("encoded_images_train_inceptionV3.p", "wb") as encoded_pickle:
    pickle.dump(encoding_train, encoded_pickle)

In [0]:
encoding_train = pickle.load(open('encoded_images_train_inceptionV3.p', 'rb'))

In [0]:
images = 'val2014/'
# Contains all the images
img = glob.glob(images+'*.jpg')
img[:5]

In [0]:
encoding_val = {}
for img in tqdm(img):
    encoding_val[img[len(images):]] = encode(img)

In [0]:
with open("encoded_images_val_inceptionV3.p", "wb") as encoded_pickle:
    pickle.dump(encoding_val, encoded_pickle)

In [0]:
#encoding_val = pickle.load(open('encoded_images_val_inceptionV3.p', 'rb'))

In [0]:
#from google.colab import files
#files.download('encoded_images_train_inceptionV3.p')
#files.download('encoded_images_val_inceptionV3.p')

In [0]:
#!ls
token = 'annotations/captions_train2014.json'

In [0]:
json_data = open(token)

In [0]:
captions = json.load(json_data)
for i, val in enumerate(captions['annotations']):
    if i<10:
        print (val)

In [0]:
for i, val in enumerate(captions['images']):
    if i<2:
        print (val)

Creating a dictionary containing all the captions of the images

In [0]:
images = 'train2014/'
img = glob.glob(images+'*.jpg')
img[:5]

In [0]:
image2id = {}

for i, val in enumerate(captions['images']):
    image2id[val['id']] = val['file_name']

In [0]:
id2captions = {}

for i, val in enumerate(captions['annotations']):
    id2captions[val['image_id']] = val['caption']

In [0]:
image2caption = {}
for key, val in id2captions.items():
    image2caption[image2id[key]] = val

In [0]:
len(image2caption)

In [0]:
image2caption['COCO_train2014_000000522971.jpg']

In [0]:
Image.open(img[0])

In [0]:
caps = []
for key, val in image2caption.items():
    caps.append(' '.join(list(val.split())))
    
caps[:5]

In [0]:
words = [i.split() for i in caps]
unique = []
for i in words:
    unique.extend(i)
unique.extend("<start>".split())
unique.extend("<end>".split())
unique.extend("<unk>".split())

In [0]:
unique = list(set(unique))
# with open("unique.p", "wb") as pickle_d:
#     pickle.dump(unique, pickle_d) 
len(unique)

In [0]:
word2idx = {val:index for index, val in enumerate(unique)}
word2idx['<start>']


In [0]:
idx2word = {index:val for index, val in enumerate(unique)}
idx2word[5303]

In [0]:
max_len = 0
for c in caps:
    c = c.split()
    if len(c) > max_len:
        max_len = len(c)
max_len

In [0]:
vocab_size = len(unique)

In [0]:
f = open('mscoco_training_dataset.txt', 'w')
f.write("image_id\tcaptions\n")

In [0]:
for key, val in image2caption.items():
    f.write(key + "\t" + "<start> " + val +" <end>" + "\n")

f.close()

In [0]:
df = pd.read_csv('mscoco_training_dataset.txt', delimiter='\t')

In [0]:
df = df.dropna()
df.head()
len(df)

In [0]:
#files.download('mscoco_training_dataset.txt')

In [0]:

c = [i for i in df['captions']]

In [0]:
imgs = [i for i in df['image_id']]

In [0]:
a = c[0]
a, imgs[0]

In [0]:
for i in a.split():
    print (i, "=>", word2idx[i])

In [0]:
samples_per_epoch = 0
for ca in c:
    samples_per_epoch = samples_per_epoch + len(ca.split())-1

In [0]:
samples_per_epoch

In [0]:
def data_generator(df, batch_size=32):
    # Shuffling the dataframe before creating batches
#     df = df.sample(frac=1)
#     c = [i for i in df['captions']]
#     imgs = [i for i in df['image_id']]
    
    partial_caps = []
    images = []
    next_words = []
    count = 0
    for i, text in enumerate(c[:1000]):
        current_image = encoding_train[imgs[i]]
        for j in range(len(text.split())-1):
            count+=1
            if text.split()[j] in word2idx:
                partial = [word2idx[text.split()[j]]]
            else:
                partial = [word2idx['<unk>']]
            partial_caps.append(partial)
            # Initializing with zeros to create a one-hot encoding matrix
            # This is what we have to predict
            # Hence initializing it with vocab_size length
            next = np.zeros(vocab_size)
            # Setting the next word to 1
            if text.split()[j+1] in word2idx:
                next[word2idx[text.split()[j+1]]] = 1
            else:
                next[word2idx['<unk>']] = 1
            next_words.append(next)
            images.append(current_image)
            
            if count >= batch_size:
                images = np.array(images)
                next_words = np.array(next_words)
                partial_caps = sequence.pad_sequences(partial_caps, maxlen=max_len, padding='post')
                yield [[images, partial_caps], next_words]
                partial_caps = []
                images = []
                next_words = []
                count = 0
    
    partial_caps = sequence.pad_sequences(partial_caps, maxlen=max_len, padding='post')
    return np.array(partial_caps)

In [0]:
for i, val in enumerate(data_generator(df)):
    if i==0:
        print (val)


partial_caps = []
next_words = []
images = []

partial_caps = data_generator([partial_caps, next_words, images],32)

In [0]:
#partial_caps.shape

In [0]:
embedding_size = 300

In [0]:
image_model = Sequential([
        Dense(embedding_size, input_shape=(2048,), activation='relu'),
        RepeatVector(1)
    ])

In [0]:
caption_model = Sequential([
        Embedding(vocab_size, embedding_size, input_length=max_len),
        LSTM(256, return_sequences=True),
        TimeDistributed(Dense(300))
    ])

In [0]:
final_model = Sequential([
        Merge([image_model, caption_model], mode='concat', concat_axis=1),
        Bidirectional(LSTM(256, return_sequences=False)),
        Dense(vocab_size),
        Activation('softmax')
    ])

In [0]:
final_model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

In [0]:
final_model.fit_generator(data_generator(df, batch_size=128), nb_epoch=1, steps_per_epoch=948248,verbose=2)